<a href="https://colab.research.google.com/github/siddharth-kale/DNA3/blob/main/DNA3_Emirates_Flight_Status_2019_2021_data_dump.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install --upgrade numpy
# !pip install --upgrade traffic

In [ ]:
from traffic.data import opensky
from traffic.core import Traffic
from datetime import timedelta,date,datetime
import pandas as pd
import random
from google.colab import drive

In [ ]:
opensky.username= ''
opensky.password= ''

In [ ]:
pd.options.display.max_rows=999

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
raw_aircraft_df = pd.read_csv('/content/gdrive/MyDrive/Projects/DNA3/Data/aircraftDatabase-2021-09.csv')

In [ ]:
emirates_fleet =raw_aircraft_df[raw_aircraft_df['owner']=='Emirates Airline']['icao24'].tolist()

In [ ]:
start_date="2021-10-29"
end_date = "2021-11-18"

sdate_dt = datetime.strptime(start_date,"%Y-%m-%d").date()
edate_dt = datetime.strptime(end_date,"%Y-%m-%d").date()

dateslist=[]

path='/content/gdrive/MyDrive/Projects/DNA3/Data/'


rand_end = sdate_dt 

while rand_end <= edate_dt:
  temp_date = rand_end+timedelta(days=30)
  rand_end = datetime(temp_date.year,temp_date.month,random.randint(1,5)).date()
  rand_start = rand_end-timedelta(days=1)
  print(datetime.strftime(rand_end,"%Y-%m-%d")+","+datetime.strftime(rand_start,"%Y-%m-%d"))
  if rand_end >edate_dt:
    break
  dateslist.append(datetime.strftime(rand_end,"%Y-%m-%d"))
  dates_df = pd.DataFrame(dateslist,columns=['End Date'])
  dates_df.to_csv(path+'dates/emiratesflights.csv')
  masterdf = pd.DataFrame(columns=['alert','altitude','callsign','geoaltitude','groundspeed','hour','icao24','last_position',
                           'latitude','longitude','onground','spi','squawk','timestamp','track','vertical_rate'])
  for k in range(0,len(emirates_fleet)):
    icao_24 = emirates_fleet[k]
    try:
      flightdf = opensky.history(start=datetime.strftime(rand_start,"%Y-%m-%d"),stop=datetime.strftime(rand_end,"%Y-%m-%d"),icao24=icao_24,date_delta = timedelta(days=1),return_flight=True)
      masterdf = masterdf.append(flightdf.data.copy(),ignore_index=True)
    except (pd.errors.ParserError,AttributeError):
      pass
  masterdf.to_csv(path+'emirates/'+datetime.strftime(rand_end,"%Y-%m-%d")+'.csv')

